In [1]:
from load_data import *
import L0_card
import numpy as np
import copy
import matplotlib.pyplot as plt

In [2]:
sample_size = 1000
batch_size = 16
seed = 0
model_name = "mnist"

In [3]:
X, w_bar, model, train_dataloader, test_dataloader, modules_to_prune = load_Xw(model_name, sample_size, batch_size, seed)

Started logging run 20220720_13-56-05_433106_64 of experiment mlpnet, saving checkpoints every 1 epoch
this is config before OrderedDict([('pruners', OrderedDict([('pruner_1', OrderedDict([('class', 'WoodburryFisherPruner'), ('epochs', [0, 2, 2]), ('weight_only', True), ('initial_sparsity', 0.0), ('target_sparsity', 0.9), ('modules', ['fc2']), ('keep_pruned', True)]))])), ('trainers', OrderedDict([('default_trainer', OrderedDict([('optimizer', OrderedDict([('class', 'SGD'), ('lr', 0.001), ('momentum', 0.5)])), ('lr_scheduler', OrderedDict([('class', 'ExponentialLR'), ('gamma', 0.9), ('epochs', [120, 1, 140])]))]))]))])
Overwrite arguments for one-shot!
this is config after OrderedDict([('pruners', OrderedDict([('pruner_1', OrderedDict([('class', 'BlockwiseWoodburryFisherPruner'), ('epochs', [0, 1, 1]), ('weight_only', True), ('initial_sparsity', 0.9), ('target_sparsity', 0.9), ('modules', ['fc1', 'fc2', 'fc3']), ('keep_pruned', True)]))])), ('trainers', OrderedDict([('default_trainer',

Also printing results@top5 as well!
Do log softmax:  True
=======Layers:=============
current_model_config {'arch': 'mlpnet', 'dataset': 'mnist', 'use_butterfly': False}
Do log softmax:  True
updated_state_dict odict_keys(['fc1._weight_mask', 'fc1._layer.weight', 'fc2._weight_mask', 'fc2._layer.weight', 'fc3._weight_mask', 'fc3._layer.weight'])
args is  Namespace(aa=False, always_eval_test=False, arch='mlpnet', aux_gpu_id=-1, batch_size=64, batched_test=False, cache_subset_ids=False, centered=False, check_grads=False, check_train_loss=False, checkpoint_freq=1, ckpt_epoch=-1, compare_globalmagni_mask=False, compare_models=False, config_path='../WoodFisher/configs/mlpnet_mnist_config_one_shot_woodburry_fisher.yaml', cpu=True, deterministic=True, device=device(type='cpu'), disable_bias=True, disable_log_soft=False, disable_train_random_transforms=False, disable_train_shuffle=False, disable_wdecay_after_prune=False, dset='mnist', dset_path='./datasets', dump_fisher_inv_mat=False, dump_grad

Constructed BlockwiseWoodburryFisherPruner with config:
    -epochs:[0, 1, 1]
    -weight_only:True
    -initial_sparsity:0.9
    -target_sparsity:0.9
    -modules:['fc1', 'fc2', 'fc3']
    -keep_pruned:True

IN BLOCK WOODBURRY
initial sparsity in args is 0.9
initial sparsity in config is 0.9
../WoodFisher/models/mlpnet.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


---computing gradients--  0
---computing gradients--  100
---computing gradients--  200
---computing gradients--  300
---computing gradients--  400
---computing gradients--  500
---computing gradients--  600
---computing gradients--  700
---computing gradients--  800
---computing gradients--  900


In [4]:
y = X@w_bar
p = w_bar.shape[0]
beta = w_bar
r = y - X@beta
sparsity = 0.9
ntot = w_bar.shape[0]
ntar = int(ntot * (1-sparsity))
print("ntar is ",ntar)
print("Shape of X is ", X.shape)

ntar is  3235
Shape of X is  (1000, 32360)


In [11]:
# sparsity list
k_list = [0.2,0.4,0.6,0.8,0.9,0.95,0.98]

# complexity is O(iter * non-zeros)
iter_list = [10,20,30,50,75,100,300]



In [12]:
# initial soluition
beta = np.copy(w_bar)
n, p = X.shape
for i in range(len(k_list)):
    k = int(p*(1-k_list[i]))
    cd_max_itr = iter_list[i]
    
    # active set size is max(k, min(p,kimp*k,buget)), here you can fix kimp=1, buget=inf
    beta, f, r, sol_time = L0_card.Heuristic_CD(y,X,beta,k,alpha=np.zeros(p),lambda1=0.001,
        lambda2=0.01, beta_tilde1=np.copy(w_bar), beta_tilde2=np.copy(w_bar), M=np.inf,
                        cd_max_itr=cd_max_itr,buget=None,kimp=1,sto_m = "sto")
    
    # update X, y and w_bar based on current solution beta here
    pass
    
    print("obj is",f,"cur sparsity is",k_list[i],"Sol time is",sol_time)
    
    

obj is 0.029101356518172625 cur sparsity is 0.2 Sol time is 1.3076162338256836
obj is 0.12107871550036534 cur sparsity is 0.4 Sol time is 1.4206759929656982
obj is 0.2884015475185811 cur sparsity is 0.6 Sol time is 1.264383316040039
obj is 0.5608207628983639 cur sparsity is 0.8 Sol time is 0.9449284076690674
obj is 0.7752349944546097 cur sparsity is 0.9 Sol time is 0.6572799682617188
obj is 0.949642508264156 cur sparsity is 0.95 Sol time is 0.40872788429260254
obj is 1.1201028181324248 cur sparsity is 0.98 Sol time is 0.5852174758911133
